<h2>Lab 1 - Indice invertido e Busca booleana<h2>

<h3>Para esse lab, vamos utilizar as seguintes bibliotecas:<h3>

In [2]:
import pandas as pd
import nltk as nltk

In [3]:
df = pd.read_csv("C:\\Users\\joao_\\Downloads\\noticias_estadao.csv")

Para pesquisar pelas palavras presentes no data frame vamos usar dois laços. O primeiro junta o título de cada documento com o conteúdo. Depois disso fazemos um split e, no segundo laço, vamos pesquisar se a palavra que queremos está dentro daquela nova string.

In [51]:
#funcao para buscar as palavras desejadas
id_news = dict()  
def busca_termo(palavra_pesquisada):
    if(palavra_pesquisada in id_news):
        return id_news
    else:
        for x in range(df.shape[0]):             #fazer disso uma funcao  usar -> df.shape[0]  
            titulo = df.loc[x,'titulo']         #pego o titulo df.loc -> provavelmente no lab será df.ix
            conteudo = df.loc[x,'conteudo']      #pego o conteudo
            newWorld = titulo +  " " +  conteudo      #concateno os dois separado por espaço
            newNova = nltk.word_tokenize(newWorld.lower())   #faço split usando nltk botando tudo para lower case
            for y in range(len(newNova)):               #faço outro for para percorrer esse novo split procurando a palavra que eu quero
                if(newNova[y] == palavra_pesquisada):  
                    if(palavra_pesquisada in id_news):
                        id_news[palavra_pesquisada].append(df.loc[x,'idNoticia'])              #posso usar o x do primeiro for por que é o mesmo número do ID
                        break;                        #parar para que eu não coloque mais do mesmo indice, caso apareça mais de uma vez
                    else:
                        id_news[palavra_pesquisada] = [df.loc[x,'idNoticia']]
                        break;    
                    
        return id_news


In [40]:
#local onde vamos fazer o AND


def buscar_and(palavra1,palavra2):
    
    iterator1 = 0
    iterator2 = 0
    
    and_answer = []
    
    first = busca_termo(palavra1)
    second = busca_termo(palavra2)
    
    id_and1 = first[palavra1]
    id_and2 = second[palavra2]
    
    while(iterator1 < len(id_and1) and iterator2 < len(id_and2)):
      if(id_and1[iterator1] == id_and2[iterator2]):
          and_answer.append(id_and1[iterator1])
          iterator1 +=1
          iterator2 +=1
      elif(id_and2[iterator2] > id_and1[iterator1]):
          iterator1 +=1
      else:
          iterator2 += 1
    return and_answer

#resultado eh 201
#assert len(search("debate AND presidencial")) 
assert1_and = buscar_and('debate','presidencial')
len(assert1_and)

201

In [14]:
#assert len(search("presidenciáveis AND corruptos")) == 0
assert2_and = buscar_and('presidenciáveis','corruptos')
len(assert2_and)

0

In [15]:
#assert len(search("Belo AND Horizonte")) == 242
assert3_and = buscar_and('belo','horizonte')
len(assert3_and)

242

In [54]:
#local onde vamos fazer o OR

def buscar_or(palavra1,palavra2):
    
    iterator1 = 0
    iterator2 = 0
    
    or_answer = []
    
    first = busca_termo(palavra1)
    second = busca_termo(palavra2)
    
    id_or1 = first[palavra1]
    id_or2 = second[palavra2]
    
    while(iterator1 < len(id_or1) and iterator2 < len(id_or2)):
          if(id_or1[iterator1] == id_or2[iterator2]):
              or_answer.append(id_or1[iterator1])
              iterator1 +=1
              iterator2 +=1
          elif(id_or2[iterator2] > id_or1[iterator1]):
              or_answer.append(id_or1[iterator1])
              iterator1 += 1
          else:
              or_answer.append(id_or2[iterator2])
              iterator2 += 1
    return or_answer

#assert len(search("debate OR presidencial")) == 1770
assert1_or = buscar_or('debate','presidencial')
len(assert1_or)

1769

In [56]:
#assert len(search("presidenciáveis OR corruptos")) == 164
assert2_or = buscar_or('presidenciáveis','corruptos')
len(assert2_or)

159

In [58]:
#assert len(search("Belo OR Horizonte")) == 331
assert3_or = buscar_or('horizonte','belo')
len(assert3_or)

330